In [26]:
import arkouda as ak
import arachne as ar
import pandas as pd
import time as time
import networkx as nx
import random


In [ ]:
ak.connect("n120", 5555)

In [ ]:


# Read the TSV file using pandas
network_df = pd.read_csv("/scratch/users/md724/DataSets/wcc/test_network.tsv", sep="\t", header=None, names=["src", "dst"])
network_df['type'] = 'T1'

# Print the pandas DataFrame (optional, just to verify)
print(network_df)
print(network_df.head())



In [ ]:
# Concatenate 'src' and 'dst' columns and find unique elements for nodes
unique_nodes = pd.concat([network_df['src'], network_df['dst']]).unique()

# Number of unique nodes
num_unique_nodes = len(unique_nodes)
num_unique_nodes

In [ ]:
# Number of edges is the length of the DataFrame
num_edges = len(network_df)

# Calculate graph density
d = num_edges / (num_unique_nodes * (num_unique_nodes - 1))
print(f"Graph Density: {d}")

print(f"Number of unique nodes: {num_unique_nodes}")
print(f"Number of edges: {num_edges}")


In [ ]:

# Convert the pandas DataFrame to an Arkouda DataFrame
ak_network_df = ak.DataFrame(network_df)
ak_network_df

In [45]:
# Create an Arachne PropGraph and load the edge attributes
ar_network_graph = ar.PropGraph()
ar_network_graph.load_edge_attributes(ak_network_df, source_column="src", destination_column="dst", relationship_columns=["type"])


In [ ]:
# Get all nodes and create node attributes
all_nodes = ak.concatenate([ak_network_df['src'], ak_network_df['dst']])
unique_nodes = ak.unique(all_nodes)
lbls = ak.array(["1"] * unique_nodes.size)
network_node_df = ak.DataFrame({"nodes": unique_nodes, "lbls": lbls})
network_node_df


In [ ]:

ar_network_graph.load_node_attributes(network_node_df, node_column="nodes", label_columns=["lbls"])
ar_network_graph

In [ ]:
isos = ar.well_connected_components(ar_network_graph)
